In [1]:
# %pip install pdf2image
# %pip install opencv-python pillow
# %pip install pytesseract

In [2]:
from pdf2image import convert_from_path
import pytesseract
import cv2
from pytesseract import Output
import pandas as pd
import numpy as np

In [3]:
def confidenceLevelSegregation(df,confLevel):
    #Lower confidence boxes shown in red
    if int(df['conf']) < confLevel:
        (x, y, w, h) = (df['left'], df['top'], df['width'], df['height'])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0,0, 255), 2) #(BGR),thickness
        df['confidenceLevel'] = 'Low'
    else: # Rest of the boxes are in green
        (x, y, w, h) = (df['left'], df['top'], df['width'], df['height'])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0,0, 255), 2) #(BGR),thickness
        df['confidenceLevel'] = 'High'
    # # Creating a visualization that showcases the boxes 
    # # cv2.imshow('img', img)
    # # cv2.waitKey(5000)
    # df['confidenceLevel'] = np.where(
    #     df['conf'] < confLevel, # condition
    #     "Low",    # result if true
    #     "High",   # result if false
    # )
    # df['confidenceLevel'] = df['conf'].map(lambda x : "Low" if(x < confLevel) else "High")
    return df

In [4]:
def paraBreakdownConfidenceLevel(folderName,pageNum,confLevel):
    img = cv2.imread(f'{folderName}/Page_{pageNum}.jpg')
    custom_config = r'--psm 3'
    d = pytesseract.image_to_data(img,config=custom_config, output_type=Output.DICT)
    # print(d.keys())
    df = pd.DataFrame(d)
    #Grouping the text by their block numbers
    processedDf = df.groupby(['block_num']).agg(
        {
            'left':min,  
            'top': min,  
            'width': max,
            'height': max,
            'conf': 'mean',
            'text': (lambda x: ' '.join(x))
        }
    )
    grouped_df = processedDf.reset_index()
    grouped_df['Page'] = pageNum

    for i in range(len(grouped_df)):
        if int(grouped_df.loc[i,'left']) < confLevel:
            (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            df['confidenceLevel'] = 'Low'
        else:
            (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, ), 2)
            df['confidenceLevel'] = 'High'
            
    # # Creating a visualization that showcases the boxes 
    # # cv2.imshow('img', img)
    # # cv2.waitKey(5000)
    return df


    # segregatedDF = confidenceLevelSegregation(grouped_df,confLevel)
    # return segregatedDF


In [5]:
def paraBreakdown(folderName,pageNum):
    img = cv2.imread(f'{folderName}/Page_{pageNum}.jpg')
    custom_config = r'--psm 3'
    d = pytesseract.image_to_data(img,config=custom_config, output_type=Output.DICT)
    # print(d.keys())
    df = pd.DataFrame(d)
    #Grouping the text by their block numbers
    processedDf = df.groupby(['block_num']).agg(
        {
            'left':min,  
            'top': min,  
            'width': max,
            'height': max,
            'conf': 'mean',
            'text': (lambda x: ' '.join(x))
        }
    )
    grouped_df = processedDf.reset_index()
    grouped_df['Page'] = pageNum
    # # Creating a visualization that showcases the boxes 
    # for i in range(len(grouped_df)):
    # # print(grouped_df.loc[i,'left'])
    #     if int(grouped_df.loc[i,'left']) > 60:
    #         (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
    #         img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)


    # cv2.imshow('img', img)
    # cv2.waitKey(5000)
    # grouped_df.to_csv(f'./Page_{pageNum}paraBlocksTrial.csv')

    # display(grouped_df)
    return grouped_df

In [6]:
#Converting pdf pages into individual images
def convertPDFtoImages(pdfPath,pdfFolderToStoreImages,outputFilePath,confLevel):
    pdfs = pdfPath #"./Articles/amz_letters.pdf"
    pages = convert_from_path(pdfs, 350)
    print('Ingested all the pages from the pdf. Moving on to process them next ==>')
    df = pd.DataFrame()
    i = 1
    for page in pages:
        image_name = "Page_" + str(i) + ".jpg"  
        page.save(f'{pdfFolderToStoreImages}/{image_name}', "JPEG")
        print(f'Doing iteration number {i}')
        res = paraBreakdownConfidenceLevel(pdfFolderToStoreImages,i,confLevel)
        # display(res)
        df=pd.concat([df,res], ignore_index=True)
        i = i+1
    display(df)
    df.to_csv(f'{outputFilePath}')

In [8]:
convertPDFtoImages('./Articles/Carbon-Credit-Carbon-Offset-Fundamentals.pdf','carbonCredit','carbonCredit/fullCarbonCreditParaBlocks.csv',30)

Ingested all the pages from the pdf. Moving on to process them next ==>
Doing iteration number 1
Doing iteration number 2
Doing iteration number 3
Doing iteration number 4
Doing iteration number 5
Doing iteration number 6
Doing iteration number 7
Doing iteration number 8
Doing iteration number 9
Doing iteration number 10
Doing iteration number 11
Doing iteration number 12
Doing iteration number 13
Doing iteration number 14
Doing iteration number 15
Doing iteration number 16
Doing iteration number 17
Doing iteration number 18
Doing iteration number 19
Doing iteration number 20
Doing iteration number 21
Doing iteration number 22
Doing iteration number 23
Doing iteration number 24
Doing iteration number 25
Doing iteration number 26
Doing iteration number 27


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,confidenceLevel
0,1,1,0,0,0,0,0,0,2975,3850,-1,,Low
1,2,1,1,0,0,0,265,228,2448,481,-1,,Low
2,3,1,1,1,0,0,265,228,2448,481,-1,,Low
3,4,1,1,1,1,0,868,228,1386,108,-1,,Low
4,5,1,1,1,1,1,868,228,702,108,95,CARBON,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12120,5,1,6,1,1,17,2382,3606,325,50,95,Washington,High
12121,2,1,7,0,0,0,2796,3382,176,468,-1,,High
12122,3,1,7,1,0,0,2796,3382,176,468,-1,,High
12123,4,1,7,1,1,0,2796,3382,176,468,-1,,High


In [9]:
convertPDFtoImages('./Articles/putting-carbon-markets-to-work-on-the-path-to-net-zero.pdf','carbonNetZero','carbonNetZero/fullCarbonNetZeroParaBlocks.csv',30)

Ingested all the pages from the pdf. Moving on to process them next ==>
Doing iteration number 1
Doing iteration number 2
Doing iteration number 3
Doing iteration number 4
Doing iteration number 5
Doing iteration number 6
Doing iteration number 7
Doing iteration number 8
Doing iteration number 9
Doing iteration number 10
Doing iteration number 11
Doing iteration number 12
Doing iteration number 13
Doing iteration number 14
Doing iteration number 15
Doing iteration number 16
Doing iteration number 17
Doing iteration number 18
Doing iteration number 19
Doing iteration number 20
Doing iteration number 21
Doing iteration number 22
Doing iteration number 23
Doing iteration number 24
Doing iteration number 25
Doing iteration number 26
Doing iteration number 27
Doing iteration number 28
Doing iteration number 29
Doing iteration number 30
Doing iteration number 31
Doing iteration number 32
Doing iteration number 33
Doing iteration number 34
Doing iteration number 35
Doing iteration number 36
D

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,confidenceLevel
0,1,1,0,0,0,0,0,0,2894,4093,-1,,High
1,2,1,1,0,0,0,197,1116,487,65,-1,,High
2,3,1,1,1,0,0,197,1116,487,65,-1,,High
3,4,1,1,1,1,0,197,1116,487,65,-1,,High
4,5,1,1,1,1,1,197,1116,304,65,96,October,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31024,5,1,5,1,2,1,371,3777,757,46,76,https://sg.linkedin.com/company/gic,High
31025,2,1,6,0,0,0,368,3815,763,8,-1,,High
31026,3,1,6,1,0,0,368,3815,763,8,-1,,High
31027,4,1,6,1,1,0,368,3815,763,8,-1,,High


In [10]:
convertPDFtoImages('./Articles/MethodologyForAssessingTheQualityOfCarbonCredits.pdf','carbonMethodology','carbonMethodology/fullcarbonMethodologyParaBlocks.csv',30)


Ingested all the pages from the pdf. Moving on to process them next ==>
Doing iteration number 1
Doing iteration number 2
Doing iteration number 3
Doing iteration number 4
Doing iteration number 5
Doing iteration number 6
Doing iteration number 7
Doing iteration number 8
Doing iteration number 9
Doing iteration number 10
Doing iteration number 11
Doing iteration number 12
Doing iteration number 13
Doing iteration number 14
Doing iteration number 15
Doing iteration number 16
Doing iteration number 17
Doing iteration number 18
Doing iteration number 19
Doing iteration number 20
Doing iteration number 21
Doing iteration number 22
Doing iteration number 23
Doing iteration number 24
Doing iteration number 25
Doing iteration number 26
Doing iteration number 27
Doing iteration number 28
Doing iteration number 29
Doing iteration number 30
Doing iteration number 31
Doing iteration number 32
Doing iteration number 33
Doing iteration number 34
Doing iteration number 35
Doing iteration number 36
D

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,confidenceLevel
0,1,1,0,0,0,0,0,0,2895,4093,-1,,High
1,2,1,1,0,0,0,2100,279,548,69,-1,,High
2,3,1,1,1,0,0,2100,279,548,69,-1,,High
3,4,1,1,1,1,0,2100,279,548,69,-1,,High
4,5,1,1,1,1,1,2100,279,417,69,91,Oko-Institut,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65076,5,1,12,1,1,1,272,1561,2347,8,95,,High
65077,2,1,13,0,0,0,2550,3880,60,31,-1,,High
65078,3,1,13,1,0,0,2550,3880,60,31,-1,,High
65079,4,1,13,1,1,0,2550,3880,60,31,-1,,High
